# Basic Concepts

## What is Mizore?

Mizore aims to provide a *programmer-friendly*, *highly-modularized* and *high-performance* software platform for *near-term* quantum algorithm developing. So far, Mizore has included three modules. One can find the introduction to them in the following notebooks.
- [Adaptive Circuit Construction](AdaptiveCircuitConstruction.ipynb)
- [Quantum Subspace Expansion](QuantumSubspaceExpansion.ipynb)
- [Correlation Network](CorrelationNetwork.ipynb)

The developers of Mizore believe that the above three modules present three of the most important ideas for near-term quantum algorithms, especially for those will have practical applications. One can read the README file in the root path for a detailed introduction.

## Introduction to VQE

Mizore's efforts are centered at VQE. Mizore provides a general framework for adaptive circuit constrcution in Variational Quantum Eigensolver (VQE). VQE is a class of quantum algorithms aiming to solve the ground state energy of a Hamiltonian. 
$\newcommand\ket[1]{|#1\rangle} \newcommand\bra[1]{\langle#1|}$

During a VQE run, a parameterized trial wavefunction $\ket{\Psi(\vec{\varphi})}=\hat{U}(\vec{\varphi})\ket{\Psi_0}$ is prepared on quantum computers. And the expectation value $E = \bra{\Psi(\vec{\varphi})}\hat H\ket{\Psi(\vec{\varphi})}$ should be minimized. By the variational law, this minimized expectation value is larger or equal to the ground state energy. If the ansatz $\hat{U}(\vec{\varphi})$ is chosen properly, the outcome of VQE can be very closed to the true ground state energy.

The essential steps of a VQE includes
- Construction of the problem Hamiltonian
    - Active space selection
    - Fermion-qubit transformation (Encoding)
    - Stationary qubit reduction
    - Penalty terms
- Construction of the ansatz
    - Building block
    - Construction method
- Parameter Optimization

For every step of VQE, Mizore provides easy-to-learn and versitile functions for the users. In the following chapter, the usage of the functions will be shown.

## Hamiltonian Construction

In Mizore the problem Hamiltonian can be easily constrcuted by the package `HamiltonianGenerator`.

In [2]:
from mizore.HamiltonianGenerator.TestHamiltonian import make_example_H2
energy_obj=make_example_H2()

Symmetry: Dooh  is used when build the molecule.


Notice that energy_obj is not a `QubitOperator`. Instead, it is a `Objective.EnergyObjective` which includes more information about the task, such as the Hartree-Fock initialization and energy to terminate the calculation.

In [3]:
print("Number of qubits:",energy_obj.n_qubit)
print("Initial block:",energy_obj.init_block)
print("Chemical accuarte energy:",energy_obj.obj_info["terminate_cost"])
print("The Hamiltonian:")
print(energy_obj.hamiltonian)

Number of qubits: 4
Initial block: Type:HartreeFockInitBlock; Para Num:0; Qsubset:[0]
Chemical accuarte energy: -1.1362838344885025
The Hamiltonian:
-0.09706626816762906 [] +
0.04530261550379932 [X0 Z1 X2] +
0.04530261550379932 [X0 Z1 X2 Z3] +
0.04530261550379932 [Y0 Z1 Y2] +
0.04530261550379932 [Y0 Z1 Y2 Z3] +
0.17141282644776887 [Z0] +
0.17141282644776895 [Z0 Z1] +
0.16592785033770358 [Z0 Z1 Z2] +
0.16592785033770358 [Z0 Z1 Z2 Z3] +
0.12062523483390425 [Z0 Z2] +
0.12062523483390425 [Z0 Z2 Z3] +
0.16868898170361218 [Z1] +
-0.22343153690813575 [Z1 Z2 Z3] +
0.17441287612261608 [Z1 Z3] +
-0.22343153690813575 [Z2]


For advanced instruction of Hamiltonian, please see [HamiltonianGenerator](HamiltonianGenerator.ipynb).

## Circuit (ansatz) constrction

### Block

The class for the building blocks of a quantum circuit is defined in *Blocks* as `Block`. A `Block` is a piece of quantum circuit that have certain physical meaning. Here we introduce some common blocks.

- The High-dimensional rotation block which implements the operation $e^{iPt}$, where $P$ is a pauli word (tensor product of pauli operators on qubits) and $t$ is an adjustable parameter. This block is commonly used for simulation the evolution of a system.

In [5]:
from mizore.Blocks import RotationEntangler
print(RotationEntangler((1,2,3),(3,2,1)))

Type:RotationEntangler; Para Num:1; Qsubset:(1, 2, 3); Pauli:ZYX


- The hardware-efficient ansatz which is a layer of full rotation on each qubit, followed by a entangling operation. Here, the entangling operation we choose is CNOT gates between adjacent qubits.

In [6]:
from mizore.Blocks import HardwareEfficientEntangler
print(HardwareEfficientEntangler((1,2,3)))

Type:HardwareEfficientEntangler; Para Num:9; Qsubset:(1, 2, 3)


By wrapping quantum operations as blocks, Mizore provides convenient inversion of a block. The block can be specified to be `INVERSED`, so that they will apply the inversed quantum operation when used.

In [7]:
block=HardwareEfficientEntangler((1,2,3))
block.is_inversed=True
print(block)

Type:HardwareEfficientEntangler; Para Num:9; INVERSED; Qsubset:(1, 2, 3)


For more instruction of Blocks, please see [Blocks](Blocks.ipynb).

### BlockCircuit

In Mizore, the Blocks are arranged by `BlockCircuit`, which contains a list of Blocks. Convienient gate count is implemented.

In [8]:
from mizore.Blocks import BlockCircuit
from mizore.Blocks import HardwareEfficientEntangler,RotationEntangler

n_qubit=4
bc=BlockCircuit(n_qubit)
bc.add_block(HardwareEfficientEntangler((0,1)))
bc.add_block(RotationEntangler((1,2,3),(3,2,1)))
bc.add_block(HardwareEfficientEntangler((2,3)))

print(bc)
print("Gate used:",bc.get_gate_used())

Block Num:3; Qubit Num:4
Block list:
Type:HardwareEfficientEntangler; Para Num:6; Qsubset:(0, 1)
Type:RotationEntangler; Para Num:1; Qsubset:(1, 2, 3); Pauli:ZYX
Type:HardwareEfficientEntangler; Para Num:6; Qsubset:(2, 3)
Gate used: {'CNOT': 8, 'SingleRotation': 9, 'TimeEvolution': 0}


In Mizore, functions are provided for usually used circuit operations. They are contained in `Blocks._utilities`.

In [9]:
from mizore.Blocks._utilities import get_inverse_circuit,concatenate_circuit
ibc=get_inverse_circuit(bc)
cbc=concatenate_circuit(bc,ibc)
print("The inverse circuit")
print(ibc)
print("The concatenated circuit")
print(cbc)

The inverse circuit
Block Num:3; Qubit Num:4
Block list:
Type:HardwareEfficientEntangler; Para Num:6; INVERSED; Qsubset:(2, 3)
Type:RotationEntangler; Para Num:1; INVERSED; Qsubset:(1, 2, 3); Pauli:ZYX
Type:HardwareEfficientEntangler; Para Num:6; INVERSED; Qsubset:(0, 1)
The concatenated circuit
Block Num:6; Qubit Num:4
Block list:
Type:HardwareEfficientEntangler; Para Num:6; Qsubset:(0, 1)
Type:RotationEntangler; Para Num:1; Qsubset:(1, 2, 3); Pauli:ZYX
Type:HardwareEfficientEntangler; Para Num:6; Qsubset:(2, 3)
Type:HardwareEfficientEntangler; Para Num:6; INVERSED; Qsubset:(2, 3)
Type:RotationEntangler; Para Num:1; INVERSED; Qsubset:(1, 2, 3); Pauli:ZYX
Type:HardwareEfficientEntangler; Para Num:6; INVERSED; Qsubset:(0, 1)


The inner product of the states produced by two circuits, and the amplitude of getting $|00\dots 00\rangle$, can be calculated by `get_inner_two_circuit_product` and `get_0000_amplitude_on_circuit`.

In [10]:
from mizore.Blocks._utilities import get_inner_two_circuit_product,get_0000_amplitude_on_circuit

print(get_inner_two_circuit_product(bc,bc)) #Should be 1 because <psi|psi>=1
print(get_0000_amplitude_on_circuit(cbc)) #Should be 1 because <0|UU-|0>=1

print(get_inner_two_circuit_product(cbc,bc))
print(get_0000_amplitude_on_circuit(bc))

(0.9999999999999999+0j)
(0.9999999999999999+0j)
(1+0j)
(1+0j)


### Optimizer

Mizore provides parameter optimizer (or updater) in its `ParameterOptimizer` package. The optimizers are designed to accept the `Cost` objects. For a ground state problem, the `Cost` object can be obtained from the `EnergyObjective` by `get_cost()`. A run of parameter optimization in Mizore is like the following.

In [11]:
from mizore.ParameterOptimizer import BasinhoppingOptimizer
bc=BlockCircuit(n_qubit)
bc.add_block(energy_obj.init_block) # Add the Hartree-Fock initialization to the circuit
bc.add_block(HardwareEfficientEntangler((0,2)))
bc.add_block(HardwareEfficientEntangler((0,2)))
print(bc)
optimizer=BasinhoppingOptimizer(random_initial=0.1) # The initial parameter will be a random value between -0.1 and +0.1
energy, amp = optimizer.run_optimization(bc,energy_obj.get_cost())
print("Energy:",energy)
print("Amplitutes", amp)

Block Num:3; Qubit Num:4
Block list:
Type:HartreeFockInitBlock; Para Num:0; Qsubset:[0]
Type:HardwareEfficientEntangler; Para Num:6; Qsubset:(0, 2)
Type:HardwareEfficientEntangler; Para Num:6; Qsubset:(0, 2)
Energy: -1.137283834487081
Amplitutes [-1.53142064e+00 -1.99394183e-05 -3.50045250e-03 -8.34175105e-02
 -8.36258903e-03  1.97455155e-05  1.61117490e+00  9.09224485e-03
 -1.77569693e-07 -2.22249586e-01  1.53041813e+00  1.52964833e+00]


Sometime, one may want to avoid laborious whole-parameter optimization and only want to carry out an optimization of some of the parameters. In Mizore, one can easily control which block to optimize by setting the `active_position_list` of a `BlockCircuit`. The optimizer will only optimize the block of index in `active_position_list`. Please run the following cell and observe the length of the result amplitude.

In [12]:
bc.active_position_list=[2] # Only adjust the parameters in the last block
optimizer=BasinhoppingOptimizer(random_initial=0.1)
energy, amp = optimizer.run_optimization(bc,energy_obj.get_cost())
print("Energy:",energy)
print("Amplitutes", amp)

Energy: -1.1372838344884957
Amplitutes [ 1.53170933e+00 -1.71348898e+00 -1.07145475e-08 -2.27922532e-01
  1.60988325e+00  1.71722401e+00]


One *must* notice that, the optimized amplitude will not store in the `BlockCircuit` automatically. Instread, one need to use the following operations to update the parameters.

In [13]:
from mizore.Blocks._utilities import get_circuit_energy
hamiltonian=energy_obj.hamiltonian
print("Energy before adjust:", get_circuit_energy(bc,hamiltonian))
bc.adjust_parameter_on_active_position(amp) #Update the parameters on the active blocks
print("Energy Now:", get_circuit_energy(bc,hamiltonian))

Energy before adjust: 0.9231791809225485
Energy Now: -1.1372838344884957


Imaginary Time Evolution (ITE) on ansatz can also be easily carried out ([npj Quantum Information (2019) 5:75](https://www.nature.com/articles/s41534-019-0187-2)) .

In [14]:
from mizore.ParameterOptimizer import ImaginaryTimeEvolutionOptimizer
bc=BlockCircuit(n_qubit)
bc.add_block(energy_obj.init_block) # Add the Hartree-Fock initialization to the circuit
bc.add_block(HardwareEfficientEntangler((0,2)))
bc.set_all_block_active() # Make every block adjustable
n_step=20
optimizer=ImaginaryTimeEvolutionOptimizer(random_adjust=0.1,verbose=True, n_step=n_step, stepsize=5e-1, max_increase_n_step=n_step,fig_path=None)
#Change fig_path to "screen" to see the graphs
energy, amp = optimizer.run_optimization(bc,energy_obj.get_cost())

Lowest History Energy: 0.4626181460271928 ; Energy Now: 0.46800153881109663
Lowest History Energy: 0.4626181460271928 ; Energy Now: 0.4643543036307477
Lowest History Energy: 0.4503459738895421 ; Energy Now: 0.4503459738895421
Lowest History Energy: 0.4503459738895421 ; Energy Now: 0.4608239207980568
Lowest History Energy: 0.4503459738895421 ; Energy Now: 0.4581970649110647
Lowest History Energy: 0.4503459738895421 ; Energy Now: 0.45760193562523266
Lowest History Energy: 0.44741999962511736 ; Energy Now: 0.44741999962511736
Lowest History Energy: 0.4471055914709037 ; Energy Now: 0.4471055914709037
Lowest History Energy: 0.4471055914709037 ; Energy Now: 0.4478924495560799
Lowest History Energy: 0.4471055914709037 ; Energy Now: 0.4510617527066188
Lowest History Energy: 0.4471055914709037 ; Energy Now: 0.45759065065570753
Lowest History Energy: 0.4471055914709037 ; Energy Now: 0.4582429471099591
Lowest History Energy: 0.4471055914709037 ; Energy Now: 0.46143383125536075
Lowest History Ener

In fact, we also provides Real Time Evolution (RTE) on ansatz as `RealTimeEvolutionOptimizer`. Though, unlike ITE, usually by RTE we cannot decrease the energy esitimation. However, we believe that ansatz-RTE is important for constructing a subspace which covers the ground state. See [Quantum Subspace Expansion](QuantumSubspaceExpansion.ipynb) for more information.

So far, we have introduced all the common concepts used by modules in Mozore. To learn more about Mizore, one may start from one of the following notebook.

- [Adaptive Circuit Construction](AdaptiveCircuitConstruction.ipynb)
- [Quantum Subspace Expansion](QuantumSubspaceExpansion.ipynb)
- [Correlation Network](CorrelationNetwork.ipynb)

<img src="mizore_icon.png" width="40%" align="left" />
